In [1]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv("HACKATHON_TRAINING_DATA.CSV")

flag_columns = ['SI_FLG', 'LOCKER_HLDR_IND', 'UID_FLG', 'KYC_FLG', 'INB_FLG', 'EKYC_FLG']
df[flag_columns] = df[flag_columns].replace({'Y': 1, 'N': 0})
df[flag_columns] = df[flag_columns].astype(float)

def convert_to_months(s):
    if pd.isna(s): return np.nan
    s = str(s).lower().strip()
    pattern = r'(?:(\d+)\s*yrs?)?\s*(?:(\d+)\s*(?:months|mon))?'
    match = re.match(pattern, s)
    if match:
        years = int(match.group(1)) if match.group(1) else 0
        months = int(match.group(2)) if match.group(2) else 0
        return years * 12 + months
    return np.nan

df['CREDIT_HISTORY_LENGTH1'] = df['CREDIT_HISTORY_LENGTH1'].apply(convert_to_months)
df['AVERAGE_ACCT_AGE1'] = df['AVERAGE_ACCT_AGE1'].apply(convert_to_months)

income_band_mapping = {
    'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7,
    'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'EX05': 14
}
df['INCOME_BAND1'] = df['INCOME_BAND1'].map(lambda x: income_band_mapping.get(x, np.nan))

if 'ONEMNTHCR' in df.columns:
    df.rename(columns={'ONEMNTHCR': 'ONEMNTHSCR'}, inplace=True)

agreg_group_mapping = {
    '#Total Auto Loan': 1,
    '#Total Xpress Credit': 2,
    '#Housing Loan': 3,
    '#Education Loan Total': 4
}
df['AGREG_GROUP'] = df['AGREG_GROUP'].map(lambda x: agreg_group_mapping.get(x, np.nan))

product_type_mapping = {
    'AUTO LOAN': 1,
    'PERSONAL LOAN': 2,
    'HOME LOAN': 3,
    'EDUCATION LOAN': 4
}
df['PRODUCT_TYPE'] = df['PRODUCT_TYPE'].map(lambda x: product_type_mapping.get(x, np.nan))

month_map = {
    'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06',
    'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'
}
def convert_time_period(val):
    if pd.isna(val): return np.nan
    match = re.match(r'([A-Z]{3})(\d{2})', str(val).upper())
    if match:
        month = month_map.get(match.group(1), '00')
        year = '20' + match.group(2)
        return int(year + month)
    return np.nan

df['TIME_PERIOD'] = df['TIME_PERIOD'].apply(convert_time_period)

rg_columns = [
    'LAST_1_YR_RG4', 'LAST_3_YR_RG4', 'LAST_1_YR_RG3', 'LAST_1_YR_RG2', 'LAST_1_YR_RG1',
    'FIRST_NPA_TENURE', 'CUST_NO_OF_TIMES_NPA', 'LATEST_NPA_TENURE', 'NO_YRS_NPA',
    'LATEST_RG3_TENURE', 'NO_YRS_RG3', 'TOT_IRAC_CHNG', 'TIMES_IRAC_SLIP', 'TIMES_IRAC_UPR',
    'NO_ENQ', 'CRIFF_11', 'CRIFF_22', 'CRIFF_33', 'CRIFF_44', 'CRIFF_55', 'CRIFF_66', 'TOTAL_CRIFF1'
]
df[rg_columns] = df[rg_columns].fillna(0)

sdr_cols = [f'{i}MNTHSDR' for i in [
    'ONE','TWO','THREE','FOUR','FIVE','SIX','SEVEN','EIGHT','NINE','TEN','ELEVEN','TWELVE'
]]
df[sdr_cols] = df[sdr_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
df['ALL_LON_LIMIT'] = pd.to_numeric(df['ALL_LON_LIMIT'], errors='coerce').fillna(0)
df[sdr_cols] = df[sdr_cols].abs()
monthly_limit = df['ALL_LON_LIMIT'] / 12
df_overspend = df[sdr_cols].sub(monthly_limit, axis=0)
total_spend = df[sdr_cols].sum(axis=1)
total_overspend = df_overspend.clip(lower=0).sum(axis=1)
df['overspend_ratio'] = total_overspend / (total_spend + 1e-6)

limit_matrix = pd.DataFrame(
    np.tile(monthly_limit.values[:, None], (1, len(sdr_cols))),
    columns=sdr_cols, index=df.index
)
overspend_flags = df[sdr_cols] > limit_matrix
def max_consecutive_true(arr):
    max_streak = streak = 0
    for val in arr:
        if val:
            streak += 1
            max_streak = max(max_streak, streak)
        else:
            streak = 0
    return max_streak
df['max_consec_overspend'] = overspend_flags.apply(max_consecutive_true, axis=1)

out_cols = [f'{i}MNTHOUTSTANGBAL' for i in [
    'TWELVE','ELEVEN','TEN','NINE','EIGHT','SEVEN',
    'SIX','FIVE','FOUR','THREE','TWO','ONE'
]]
df[out_cols] = df[out_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
def calc_slope(row):
    x = np.arange(12)
    y = row.values
    return np.polyfit(x, y, 1)[0]
df['outbal_slope'] = df[out_cols].apply(calc_slope, axis=1)
df['outbal_is_declining'] = (df['outbal_slope'] < 0).astype(int)

term_debit_cols = [f"{i}MNTHAVGMTD" for i in [
    'ONE','TWO','THREE','FOUR','FIVE','SIX','SEVEN',
    'EIGHT','NINE','TEN','ELEVEN','TWELVE'
]]
df[term_debit_cols] = df[term_debit_cols].fillna(0)
df['slope_MTD'] = df[term_debit_cols].apply(calc_slope, axis=1)
df['is_debit_declining_MTD'] = (df['slope_MTD'] < 0).astype(int)

keywords_to_remove = ['SDR', 'SCR', 'OUTSTANGBAL', 'AVGMTD', 'AVGQTD', 'AVGYTD']
exceptions = ['KYC_SCR']
cols_to_drop = [
    col for col in df.columns
    if any(kw in col for kw in keywords_to_remove) and col not in exceptions
]
df.drop(columns=cols_to_drop, inplace=True)

print(f"\nDropped {len(cols_to_drop)} columns: {cols_to_drop}")

df.to_csv("cleaned_data.csv", index=False)


C:\Users\vansh\AppData\Local\Temp\ipykernel_60264\4145292772.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[flag_columns] = df[flag_columns].replace({'Y': 1, 'N': 0})



Dropped 72 columns: ['ONEMNTHSCR', 'ONEMNTHSDR', 'ONEMNTHOUTSTANGBAL', 'ONEMNTHAVGMTD', 'ONEMNTHAVGQTD', 'ONEMNTHAVGYTD', 'TWOMNTHSCR', 'TWOMNTHSDR', 'TWOMNTHOUTSTANGBAL', 'TWOMNTHAVGMTD', 'TWOMNTHAVGQTD', 'TWOMNTHAVGYTD', 'THREEMNTHSCR', 'THREEMNTHSDR', 'THREEMNTHOUTSTANGBAL', 'THREEMNTHAVGMTD', 'THREEMNTHAVGQTD', 'THREEMNTHAVGYTD', 'FOURMNTHSCR', 'FOURMNTHSDR', 'FOURMNTHOUTSTANGBAL', 'FOURMNTHAVGMTD', 'FOURMNTHAVGQTD', 'FOURMNTHAVGYTD', 'FIVEMNTHSCR', 'FIVEMNTHSDR', 'FIVEMNTHOUTSTANGBAL', 'FIVEMNTHAVGMTD', 'FIVEMNTHAVGQTD', 'FIVEMNTHAVGYTD', 'SIXMNTHSCR', 'SIXMNTHSDR', 'SIXMNTHOUTSTANGBAL', 'SIXMNTHAVGMTD', 'SIXMNTHAVGQTD', 'SIXMNTHAVGYTD', 'SEVENMNTHSCR', 'SEVENMNTHSDR', 'SEVENMNTHOUTSTANGBAL', 'SEVENMNTHAVGMTD', 'SEVENMNTHAVGQTD', 'SEVENMNTHAVGYTD', 'EIGHTMNTHSCR', 'EIGHTMNTHSDR', 'EIGHTMNTHOUTSTANGBAL', 'EIGHTMNTHAVGMTD', 'EIGHTMNTHAVGQTD', 'EIGHTMNTHAVGYTD', 'NINEMNTHSCR', 'NINEMNTHSDR', 'NINEMNTHOUTSTANGBAL', 'NINEMNTHAVGMTD', 'NINEMNTHAVGQTD', 'NINEMNTHAVGYTD', 'TENMNTHSCR', '